In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from EM_dataset import EM_Data, EM_Sequence
from keras.models import Model
from keras.layers import Input
from keras.callbacks import CSVLogger, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras import backend as K
from keras.optimizers import RMSprop, Adadelta, Nadam
from keras.models import load_model
from keras.utils import multi_gpu_model
from keras.metrics import categorical_accuracy

import os
import my_model

Using TensorFlow backend.


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1, 2, 3"  # 要放到下面的前面

In [4]:
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
K.set_session(tf.Session(config=config))

In [5]:
K.image_dim_ordering()

'th'

In [6]:
# my version still needs to learn
def soft_max_loss(y_true, y_pred):
    y_pred_l = K.log(K.clip(y_pred, K.epsilon(),1-K.epsilon()))
    y_cal = y_true * y_pred_l  # dot is matrix mul
    y_cal_max = K.sum(y_cal, axis=2)
    return -K.sum(y_cal_max) / K.cast(K.prod([256, 256, 7]), dtype='float32') 

In [7]:
def soft_max_loss(y_true, y_pred):
    y_cal = y_true * y_pred
    y_cal_max = K.sum(y_cal, axis=2)
    return -K.sum(K.log(K.clip(y_cal_max, K.epsilon(),1-K.epsilon()))) \
                    /K.cast(K.prod([2, 7, 1, 256, 256]), dtype='float32')  # divided by size

In [8]:
def myacc(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    return K.mean(K.equal(y_true_f, K.round(y_pred_f)))

In [9]:
model_dict = {}
models = {}

model_dict['deepEM3D'] = ['deepEM3D', my_model.timeDist_DeepEM3D_Net]
models['deepEM3D'] = 'deepEM3D'
model_name = 'deepEM3D'

# model_dict[model][1] return a function pointer for creating network model
model_name = model_dict[model_name][0]
model_create_func = model_dict[model_name][1]

In [10]:
row = 256
col = 256
depth = 7
params = {'patch_shape': (256,256,7),
          'batch_size': 16,
          'data_dir': 'DATA/CREMI/',
          'data_file': 'A_train_norm.h5'}

train_gen = EM_Sequence(**params)
valid_gen = EM_Sequence(**params, train_flag=False,)

loading data file from: DATA/CREMI/A_train_norm.h5
loading data file from: DATA/CREMI/A_train_norm.h5


In [11]:
class ModelMGPU(Model):
    def __init__(self, ser_model, gpus):
        pmodel = multi_gpu_model(ser_model, gpus)
        self.__dict__.update(pmodel.__dict__)
        self._smodel = ser_model

    def __getattribute__(self, attrname):
        '''Override load and save methods to be used from the serial-model. The
        serial-model holds references to the weights in the multi-gpu model.
        '''
        # return Model.__getattribute__(self, attrname)
        if 'load' in attrname or 'save' in attrname:
            return getattr(self._smodel, attrname)

        return super(ModelMGPU, self).__getattribute__(attrname)


In [12]:
inputs = Input(shape=(None, 1, row, col)) 
outputs = model_create_func(inputs)
model = Model(inputs=inputs, outputs=outputs)

In [13]:
exp = 'cremi2'
weight_h5_file = exp + '-{epoch:02d}-{val_myacc:.2f}.h5'  # here use val_myacc
folder = './logs/' + exp + '/'

csv_logger = CSVLogger(folder + exp +'.log') #, append=True) 
# early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
best_model = ModelCheckpoint(folder + weight_h5_file, verbose=1, 
                             save_best_only=True, save_weights_only=True, period=5) 
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, 
                              verbose=1, mode='min', min_lr=1e-7)

In [14]:
tb_logger = TensorBoard(log_dir=folder, write_graph=True, update_freq='epoch',
                        )
# histogram_freq=2, write_grads=True, write_images=True
# won't work? basically because net archi, too much computation?

In [15]:
%%time

parallel_model = ModelMGPU(model, gpus=4)

CPU times: user 11.4 s, sys: 256 ms, total: 11.7 s
Wall time: 11.7 s


In [16]:
# 1e-3 will make acc down to 0
optimizer = RMSprop(lr=1e-5)

In [17]:
# this one is just not working here?
optimizer = Adadelta(lr=1e-3, rho=0.95, epsilon=None, decay=0.0)

In [18]:
# All parameter gradients will be clipped to clipnorm
# a maximum norm of 1.

# All parameter gradients will be clipped to clipvalue
# a maximum value of 0.5 and
# a minimum value of -0.5.

optimizer = Nadam(lr=0.02, beta_1=0.9, beta_2=0.999, 
                  epsilon=None, schedule_decay=0.004)

In [19]:
parallel_model.compile(optimizer = optimizer, loss = soft_max_loss, metrics = [myacc])
# parallel_model.summary()

In [ ]:
history = parallel_model.fit_generator(generator=train_gen,
                              validation_data=valid_gen, 
                              steps_per_epoch=400, epochs=500, # 1024x1024x100 / 256x256x7
                              validation_steps=50, 
                              verbose=1, workers=6, use_multiprocessing=True,
                              callbacks = [best_model, csv_logger, reduce_lr, tb_logger])
#tensorboard --logdir=/full_path_to_your_logs

Epoch 1/500
319/400 [======================>.......] - ETA: 1:18 - loss: 4.6350 - myacc: 0.7989